In [33]:
import random
import copy
import os

In [34]:
import subprocess
BLENDER_PATH="C:\\Program Files\\Blender Foundation\\Blender 3.3\\blender.exe"

### Importing function for generating chromosome

In [35]:
from CGen import *

In [73]:
L,B,H=7,6,13

In [39]:
%%time

P1=[]

for _ in range(10):
    CH=generate_chromosome(L,B,H,random.random())
    P1.append(CH)
    get_stats(CH)
    print('-'*125)

(R,G,B,A) = (236,245,217,5)
Roughness Factor = 5
Number of vertices = 6
Number of faces = 9
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (173,50,65,40)
Roughness Factor = 5
Number of vertices = 5
Number of faces = 2
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (63,182,138,75)
Roughness Factor = 5
Number of vertices = 7
Number of faces = 9
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (236,172,181,110)
Roughness Factor = 1
Number of vertices = 6
Number of faces = 11
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (33,57,115,41)
Roughness Factor = 3
Number of vertices = 7
Number of faces = 11
----------------------------------

In [65]:
%%time
for i,C in enumerate(P1):
#     R,G,B,A=C[0],C[1],C[2],C[3]
#     HEX_VALUE="#{:02x}{:02x}{:02x}{:02x}".format(R,G,B,A)
    COLOUR=str((C[0]/255,C[1]/255,C[2]/255,C[3]))
    VERTS=C[5][0]
    FACES=C[5][1]
    RO=C[4]
    FILE_NUM=str(i)
    
    #print(subprocess.run(["blender", "--background", "--python","generatePopulation.py",FILE_NUM,COLOUR,str(VERTS),str(FACES),str(RO)],capture_output=True,shell=True))

    subprocess.run(["blender", "--background", "--python","generatePopulation.py",FILE_NUM,COLOUR,str(VERTS),str(FACES),str(RO)],capture_output=True,shell=True)
    
    #os.system(f"blender --background --python generatePopulation.py {FILE_NUM} {COLOUR} {str(VERTS)} {str(FACES)} {str(RO)}")
    #print(f"blender --background --python generatePopulation.py {FILE_NUM} {COLOUR} {str(VERTS)} {str(FACES)} {str(RO)}")
    #print("  ")
    #print("  ")
    print(f'Statue {i} Saved!')
    #print("-"*125)

    
    
    

Statue 0 Saved!
Statue 1 Saved!
Statue 2 Saved!
Statue 3 Saved!
Statue 4 Saved!
Statue 5 Saved!
Statue 6 Saved!
Statue 7 Saved!
Statue 8 Saved!
Statue 9 Saved!
Wall time: 42.4 s


## Mutation

In [41]:
def mutate(C,L,B,H):
    '''
    C -> Chromosome
    L,B,H -> Length, Breadth and Height of the bounding frame
    '''
    mutated_C=copy.copy(C)
    for i in range(len(mutated_C)):
        # generating a random value and checking for threshold value
        if random.random()>0.5:
            
            # for colours R,G,B
            if i==0 or i==1 or i==2:
                mutated_C[i]=random.randint(0,255)
            
            #for alpha and roughness factor
            elif i==3 or i==4:
                mutated_C[i]=random.uniform(0,1)
            
            #for vertices and faces
            elif i==5:
                verts=generate_vertices(L,B,H,random.random())
                faces=generate_faces(len(verts),random.random())
                mutated_C[i]=[verts,faces]
    
    return mutated_C

In [42]:
get_stats(P1[0])

(R,G,B,A) = (236,245,217,5)
Roughness Factor = 5
Number of vertices = 6
Number of faces = 9


In [43]:
get_stats(mutate(P1[0],L,B,H))

(R,G,B,A) = (236,245,22,5)
Roughness Factor = 0.3172803044450633
Number of vertices = 6
Number of faces = 9


## Crossover

In [44]:
def crossover(C1,C2):
    
    c1=copy.copy(C1)
    c2=copy.copy(C2)
    
    # Choose a random crossover point
    crossover_point = random.randint(0, len(c1))
    
    for i in range(crossover_point,len(c1)):
        c1[i], c2[i] = c2[i], c1[i]
    
    return c1,c2

In [45]:
o1,o2=crossover(P1[0],P1[1])

In [46]:
get_stats(P1[0])
print('-'*125)
get_stats(P1[1])

(R,G,B,A) = (236,245,217,5)
Roughness Factor = 5
Number of vertices = 6
Number of faces = 9
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (173,50,65,40)
Roughness Factor = 5
Number of vertices = 5
Number of faces = 2


In [47]:
get_stats(o1)
print('-'*125)
get_stats(o2)

(R,G,B,A) = (236,245,65,40)
Roughness Factor = 5
Number of vertices = 5
Number of faces = 2
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (173,50,217,5)
Roughness Factor = 5
Number of vertices = 6
Number of faces = 9


## Fitness Function

In [67]:
import zlib,sys

MODELS_PATH="E:\\Research\\Statue Generator\\Rough Work\\Test"

In [71]:
def fitness(PATH):
    
    fitness=[]
    
    for file in os.listdir(PATH):
    
        with open(PATH+"\\"+file, mode="rb") as fin, open(PATH+"\\"+file[:-3]+"_compressed", mode="wb") as fout:
            data = fin.read()
            compressed_data = zlib.compress(data, zlib.Z_BEST_COMPRESSION)
            orig=sys.getsizeof(data)
            comp=sys.getsizeof(compressed_data)
            fitness.append(comp/orig)
        
    return fitness

In [72]:
fitness(MODELS_PATH)

[0.11801559949653839,
 0.11789659085218314,
 0.11801374151835356,
 0.11801234653804978,
 0.11805059660044555,
 0.11809032043953523,
 0.11787144694557328,
 0.11789739586959681,
 0.11808642118401194]